# <font color="red"> MBA em IA e Big Data</font>
## <span style="color:red">Curso 2 - Ciência de Dados, Aprendizado de Máquina e Mineração de Dados</span>

### <span style="color:darkred">Mineração de Dados</span>

#### Material de Apoio - Experimentos com Dados não Estruturados - Textos (ver Vídeo)

*Ricardo M. Marcacini*<br>
*ICMC/USP São Carlos*

# Importando bibliotecas

In [3]:
import pandas as pd
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Carregando base de dados


Dmoz-Sports-500 are composed by web pages of the sports category extracted from DMOZ - Open Directory Project. The document class are the subcateries of the sports category. We consider 500 document for each class.


Rossi, Rafael Geraldeli, Ricardo Marcondes Marcacini, and Solange Oliveira Rezende. "Benchmarking text collections for classification and clustering tasks.". University of São Paulo. Technical Report. (2013). 
http://repositorio.icmc.usp.br//handle/RIICMC/6641

In [4]:
df = pd.read_csv('Dmoz-Sports.csv')
df

,file_name,text,class
0,3191005.txt,"Bryant Bulldogs Official site with news, sched...",Baseball
1,3193010.txt,Hirschbeck v. Alomar Covers a legal case betwe...,Baseball
2,3190744.txt,Mountaineers Baseball Official site. Includes ...,Baseball
3,3193695.txt,"M.S.H. Little League Contains news items, a li...",Baseball
4,3193486.txt,Quotations: Bob Uecker Baseball Almanac collec...,Baseball
...,...,...,...
13495,3274147.txt,"IMDb.com - Dwayne Johnson Filmography, awards,...",Wrestling
13496,3273536.txt,"Suwannee Wrestling Includes pictures, and stat...",Wrestling
13497,3274032.txt,"Jeff's Swanton Biography, facts and wallpapers.",Wrestling
13498,3274152.txt,Roddy Piper Tribute Includes background inform...,Wrestling


In [5]:
print(df['class'].unique())
print(len(df['class'].unique()))

['Baseball' 'Basketball' 'Bowling' 'Cricket' 'Cycling' 'Equestrian'
 'Fencing' 'Flying' 'Football' 'Golf' 'Gymnastics' 'Hockey' 'Lacrosse'
 'Martial' 'Motorsports' 'Paintball' 'Running' 'Skating' 'Soccer'
 'Softball' 'Strength' 'Tennis' 'Track' 'Volleyball' 'Water' 'Winter'
 'Wrestling']
27


# Pré-processamento dos Textos

Vamos usar um modelo pré-treinado para extrair características dos textos.

Veja os modelos pré-treinados disponíveis na biblioteca proposta (sentence-transformers): https://www.sbert.net/docs/pretrained_models.html


In [6]:
text_feature_extractor = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/402 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

### Testando a representação

In [7]:
texto1 = "A menina chutou a bola no gol"
texto2 = "Crianças gostam de praticar esportes"
texto3 = "A inflação está aumentando"
texto4 = "The girl plays football"

dataset = [texto1, texto2, texto3, texto4]
dataset

['A menina chutou a bola no gol',
 'Crianças gostam de praticar esportes',
 'A inflação está aumentando',
 'The girl plays football']

In [8]:
X = text_feature_extractor.encode(dataset)
X

array([[-0.04990399, -0.10120969, -0.01776708, ...,  0.08765166,
         0.10532034, -0.02653974],
       [-0.1827991 , -0.14581765, -0.01475725, ...,  0.22304882,
         0.2835558 , -0.0597906 ],
       [-0.10946287, -0.02651667, -0.01413278, ..., -0.05574905,
        -0.13470888, -0.00053339],
       [-0.1311327 ,  0.05632214, -0.0117345 , ...,  0.18818966,
         0.12022091, -0.06361013]], dtype=float32)

In [9]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.049904,-0.101210,-0.017767,0.040031,0.163654,-0.112286,-0.152568,0.158626,-0.150821,0.055745,...,0.025651,0.164021,-0.198006,-0.093803,0.001125,0.245816,0.088471,0.087652,0.105320,-0.026540
1,-0.182799,-0.145818,-0.014757,0.171981,0.053201,-0.072936,-0.022060,0.049558,0.082029,-0.107070,...,0.001167,0.028555,-0.169803,-0.126898,-0.101996,0.204959,0.046652,0.223049,0.283556,-0.059791
2,-0.109463,-0.026517,-0.014133,0.099378,0.007704,0.154726,-0.134314,0.031381,-0.006636,0.214759,...,0.007699,0.041511,-0.039930,-0.067037,0.101316,-0.232816,0.092790,-0.055749,-0.134709,-0.000533
3,-0.131133,0.056322,-0.011734,-0.025238,0.137596,-0.031775,-0.206477,0.239817,-0.081239,-0.039377,...,0.003737,0.187751,-0.059036,-0.078171,-0.072765,0.329974,0.070836,0.188190,0.120221,-0.063610


In [10]:
for i in range(0, len(X)-1):
  for j in range(i+1,len(X)):
    print('Similaridade entre A e B')
    print('A = ',dataset[i])
    print('B = ',dataset[j])
    print(util.cos_sim(X[i], X[j]).numpy()[0][0])
    print('-------')

Similaridade entre A e B
A =  A menina chutou a bola no gol
B =  Crianças gostam de praticar esportes
0.45085272
-------
Similaridade entre A e B
A =  A menina chutou a bola no gol
B =  A inflação está aumentando
0.068717934
-------
Similaridade entre A e B
A =  A menina chutou a bola no gol
B =  The girl plays football
0.63144743
-------
Similaridade entre A e B
A =  Crianças gostam de praticar esportes
B =  A inflação está aumentando
0.08299054
-------
Similaridade entre A e B
A =  Crianças gostam de praticar esportes
B =  The girl plays football
0.43461877
-------
Similaridade entre A e B
A =  A inflação está aumentando
B =  The girl plays football
0.046365224
-------


## Pré-processando a base de textos

In [11]:
X = text_feature_extractor.encode(df.text.to_list())

In [12]:
df['text_features'] = list(X)
df

,file_name,text,class,text_features
0,3191005.txt,"Bryant Bulldogs Official site with news, sched...",Baseball,"[-0.036046587, 0.13251913, -0.0038513546, 0.05..."
1,3193010.txt,Hirschbeck v. Alomar Covers a legal case betwe...,Baseball,"[0.003961856, -0.19437294, -0.008163866, 0.146..."
2,3190744.txt,Mountaineers Baseball Official site. Includes ...,Baseball,"[-0.1350075, 0.12867971, 0.0032569494, 0.19287..."
3,3193695.txt,"M.S.H. Little League Contains news items, a li...",Baseball,"[-0.18122779, -0.03957395, -0.004045443, 0.156..."
4,3193486.txt,Quotations: Bob Uecker Baseball Almanac collec...,Baseball,"[-0.036253452, -0.009609588, -0.001147742, 0.0..."
...,...,...,...,...
13495,3274147.txt,"IMDb.com - Dwayne Johnson Filmography, awards,...",Wrestling,"[-0.056194462, 0.3082657, -0.0043185647, 0.046..."
13496,3273536.txt,"Suwannee Wrestling Includes pictures, and stat...",Wrestling,"[-0.07753967, 0.25586662, -0.002476903, 0.0779..."
13497,3274032.txt,"Jeff's Swanton Biography, facts and wallpapers.",Wrestling,"[-0.010930059, 0.26063913, -0.006104652, 0.083..."
13498,3274152.txt,Roddy Piper Tribute Includes background inform...,Wrestling,"[-0.14966059, 0.21381713, -0.0061597005, 0.144..."


# Experimentando um classificador k-NN

Para o nosso teste (simples), vamos dividir o conjunto de dados em 80% para treinamento e 20% para teste.

In [13]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [14]:
df_train

,file_name,text,class,text_features
12745,Sports_3272758.txt,Perfect North Slopes Located just west of Cinc...,Winter,"[-0.2771798, -0.014207737, -0.0017138981, 0.13..."
2450,3203677.txt,Flatland Place Old school tricks and flatland ...,Cycling,"[0.008829312, -0.13896446, -4.4838565e-05, 0.0..."
8261,3247702.txt,Lawrence University Cross Country Vikings. Coa...,Running,"[-0.16413541, 0.25168875, 0.0012479315, 0.1028..."
7058,3243555.txt,"Fairmont Raceway This 1/2 mile, high banked, d...",Motorsports,"[-0.09343644, -0.060069505, -0.0010917239, 0.1..."
6141,3235572.txt,Clarkson University Women's Lacrosse Golden Kn...,Lacrosse,"[-0.017799444, 0.16569644, -0.0055210395, 0.09..."
...,...,...,...,...
5191,3230716.txt,"Arizona Olympian Gymnastics Offers preschool, ...",Gymnastics,"[-0.23215884, -0.104245715, -0.0072811744, 0.1..."
13418,3273956.txt,"It's Showtime Includes articles, news, picture...",Wrestling,"[-0.12814336, 0.10038537, -0.0051835612, 0.017..."
5390,3231189.txt,Manitoba Gymnastics Association Provides infor...,Gymnastics,"[-0.20877513, 0.14891854, -0.0034147538, 0.179..."
860,3195572.txt,Saginaw Valley Men's Basketball Cardinals. Sch...,Basketball,"[-0.20442742, 0.15170267, -0.0019212218, 0.092..."


In [15]:
df_test

,file_name,text,class,text_features
1483,3200794.txt,Bomaderry Bowling Club Devoted to enjoyment an...,Bowling,"[-0.086449936, 0.22722998, -0.00022814874, 0.0..."
12892,Sports_3273255.txt,SnowboardSeattle.com Articles on backcountry s...,Winter,"[-0.148869, 0.2836484, -0.0017952527, 0.170473..."
1175,3201211.txt,W.D. Tournaments Woody Demma and several Chica...,Bowling,"[-0.12716374, -0.009309612, -0.0010912316, 0.1..."
6554,Arts_3237795.txt,Northern Karate Schools Blending the tradition...,Martial,"[-0.014099523, 0.25415033, 0.0003747005, 0.142..."
11039,and_Field_3265475.txt,North Carolina Central University - Track and ...,Track,"[-0.1598681, 0.080623664, 0.0032643853, 0.0919..."
...,...,...,...,...
10661,3264759.txt,The Official Lleyton Hewitt Web Page Official ...,Tennis,"[0.03826355, 0.22084396, -0.0010818656, 0.1811..."
13070,3273838.txt,WWE Corpbiz Official site of the parent compan...,Wrestling,"[0.0006317993, 0.05516456, -0.0026034438, 0.08..."
6243,3235362.txt,Curry College Men's Lacrosse Colonels. Provide...,Lacrosse,"[-0.07501134, 0.26612765, -0.00028457347, 0.07..."
3107,3216945.txt,"Tyshler Fencing School Location, cost, coach b...",Fencing,"[-0.007180855, 0.19359075, 0.0005010359, 0.164..."


### Instanciando o K-NN

In [16]:
knn = KNeighborsClassifier(n_neighbors=3, metric='cosine')
knn.fit(df_train['text_features'].to_list(), df_train['class'].to_list())

KNeighborsClassifier(metric='cosine', n_neighbors=3)

In [17]:
preds = knn.predict(df_test['text_features'].to_list())
df_test['preds'] = preds
df_test

,file_name,text,class,text_features,preds
1483,3200794.txt,Bomaderry Bowling Club Devoted to enjoyment an...,Bowling,"[-0.086449936, 0.22722998, -0.00022814874, 0.0...",Bowling
12892,Sports_3273255.txt,SnowboardSeattle.com Articles on backcountry s...,Winter,"[-0.148869, 0.2836484, -0.0017952527, 0.170473...",Winter
1175,3201211.txt,W.D. Tournaments Woody Demma and several Chica...,Bowling,"[-0.12716374, -0.009309612, -0.0010912316, 0.1...",Gymnastics
6554,Arts_3237795.txt,Northern Karate Schools Blending the tradition...,Martial,"[-0.014099523, 0.25415033, 0.0003747005, 0.142...",Martial
11039,and_Field_3265475.txt,North Carolina Central University - Track and ...,Track,"[-0.1598681, 0.080623664, 0.0032643853, 0.0919...",Track
...,...,...,...,...,...
10661,3264759.txt,The Official Lleyton Hewitt Web Page Official ...,Tennis,"[0.03826355, 0.22084396, -0.0010818656, 0.1811...",Hockey
13070,3273838.txt,WWE Corpbiz Official site of the parent compan...,Wrestling,"[0.0006317993, 0.05516456, -0.0026034438, 0.08...",Wrestling
6243,3235362.txt,Curry College Men's Lacrosse Colonels. Provide...,Lacrosse,"[-0.07501134, 0.26612765, -0.00028457347, 0.07...",Lacrosse
3107,3216945.txt,"Tyshler Fencing School Location, cost, coach b...",Fencing,"[-0.007180855, 0.19359075, 0.0005010359, 0.164...",Fencing


In [18]:
print(classification_report(df_test['class'].to_list(), df_test['preds'].to_list()))

              precision    recall  f1-score   support

    Baseball       0.50      0.68      0.57       114
  Basketball       0.66      0.89      0.76       102
     Bowling       0.92      0.89      0.90       101
     Cricket       0.77      0.94      0.85        90
     Cycling       0.88      0.92      0.90       107
  Equestrian       0.90      0.87      0.89        95
     Fencing       0.76      0.94      0.84       111
      Flying       0.77      0.59      0.67        97
    Football       0.59      0.41      0.49       107
        Golf       0.87      0.96      0.91        94
  Gymnastics       0.83      0.93      0.88        95
      Hockey       0.83      0.57      0.68       101
    Lacrosse       0.70      0.89      0.78       100
     Martial       0.93      0.93      0.93        88
 Motorsports       0.84      0.79      0.82       101
   Paintball       0.88      0.90      0.89        84
     Running       0.82      0.83      0.83       108
     Skating       0.88    

# Usando o classificador para novos textos


In [19]:
knn_all = KNeighborsClassifier(n_neighbors=3, metric='cosine')
knn_all.fit(df['text_features'].to_list(), df['class'].to_list())

KNeighborsClassifier(metric='cosine', n_neighbors=3)

In [20]:
texto_entrada = 'A primeira corrida vencida por Senna na Fórmula 1 foi em 21 de abril de 1985'
features = text_feature_extractor.encode([texto_entrada])
knn_all.predict(features)

array(['Motorsports'], dtype='<U11')

In [21]:
texto_entrada = 'O Flamengo venceu o campeonato brasileiro'
features = text_feature_extractor.encode([texto_entrada])
knn_all.predict(features)

array(['Soccer'], dtype='<U11')